### You are given a portfolio value function:

$$V(S_1, S_2) = w_1 S_1 + w_2 S_2 + \phi(S_1, S_2)$$

**where:**

* $S_1, S_2$: asset prices 
* $w_1 = 100, w_2 = 80$: Number of shares of stocks held in portfolio
* $\phi(S_1, S_2) = 50 \sin \left(\frac{S_1}{20}\right) \cos \left(\frac{S_2}{20}\right)$ is a nonlinear interaction term


**In this project I simply analyze this given portfolio**

In [50]:
import numpy as np

def portfolio_value(s1,s2,w1=100,w2=80,k=50):

    linear_part= w1*s1 + w2*s2
    phi=k*np.sin(s1/20)*np.cos(s2/20)
    v = linear_part + phi
    return v

# Example usage
portfolio_value(120,80)

np.float64(18409.13190789841)

**Evaluation on a Grid**

To visualize a surface, I evaluated the function on a grid of (S1, S2) values.

In [51]:
def eval_grid(s1center, s2center, range_size=40, num_points=50):
    s1_values=np.linspace(s1center-range_size,s1center+range_size,num_points)
    s2_values=np.linspace(s2center-range_size,s2center+range_size,num_points)

    s1_grid,s2_grid=np.meshgrid(s1_values,s2_values,indexing='ij')
    v_grid=portfolio_value(s1_grid,s2_grid)
    return s1_grid,s2_grid,v_grid

def portfolio_value(s1,s2,w1=100,w2=80,k=50):

    linear_part= w1*s1 + w2*s2
    phi=k*np.sin(s1/20)*np.cos(s2/20)
    v = linear_part + phi
    return v

s1,s2,v=eval_grid(100,100)
print(f"Grid shape: {v.shape}")
print(f"V range: {v.min():.0f} to {v.max():.0f}")

Grid shape: (50, 50)
V range: 10793 to 25225


### Local Behavior at a Point

For the purposes of this project I chose point (S1,S2)=(100,100)

**Partial Derivative**

The partial derivative $\frac{\partial V}{\partial S_1}$ measures how $V$ changes when **only $S_1$ changes**, holding $S_2$ fixed.

$V = w_1 S_1 + w_2 S_2 + k \sin(S_1/20) \cos(S_2/20)$:

$$\frac{\partial V}{\partial S_1} = w_1 + \frac{k}{20} \cos \left(\frac{S_1}{20}\right) \cos \left(\frac{S_2}{20}\right)$$

$$\frac{\partial V}{\partial S_2} = w_2 - \frac{k}{20} \sin \left(\frac{S_1}{20}\right) \sin \left(\frac{S_2}{20}\right)$$



In [52]:
def compute_partial_derivatives(s1, s2, w1=100, w2=80, k=50):
  dV_dS1 = w1+(k/20)*np.cos(s1/20)*np.cos(s2/20)  
  dV_dS2 = w2-(k/20)*np.sin(s1/20)*np.sin(s2/20)  
  return dV_dS1, dV_dS2

# Example usage
dV1, dV2 = compute_partial_derivatives(100, 100)
print(f"∂V/∂S₁ at (100,100) = {dV1:.4f}")
print(f"∂V/∂S₂ at (100,100) = {dV2:.4f}")
print(f"If S₁ increases by 1, V changes by approximately {dV1:.4f}")
print(f"If S₂ increases by 1, V changes by approximately {dV2:.4f}")

∂V/∂S₁ at (100,100) = 100.2012
∂V/∂S₂ at (100,100) = 77.7012
If S₁ increases by 1, V changes by approximately 100.2012
If S₂ increases by 1, V changes by approximately 77.7012


### The Directional Derivative

Unit vector $\mathbf{u} = (u_1, u_2)$

$$D_{\mathbf{u}}V = \nabla V \cdot \mathbf{u} = \frac{\partial V}{\partial S_1}u_1 + \frac{\partial V}{\partial S_2}u_2$$

This tells the rate of change of $V$ when you move in direction $\mathbf{u}$.

In [53]:
def directional_derivative(s1, s2, direction, w1=100,w2=80,k=50):
    d1,d2=direction
    magnitude=np.sqrt(d1**2 + d2**2)
    u1=d1/magnitude
    u2=d2/magnitude
    dV_dS1 = w1+(k/20)*np.cos(s1/20)*np.cos(s2/20)  
    dV_dS2 = w2-(k/20)*np.sin(s1/20)*np.sin(s2/20)  
    D_u_V= dV_dS1*u1 + dV_dS2*u2
    return D_u_V

result=directional_derivative(100,100,(1,1))
print(f"D_u V in direction (1,1) = {result:.4f}")

D_u V in direction (1,1) = 125.7959


In [54]:
def analyze_directions(s1, s2):
  directions = [
      ("East", (1, 0)),
      ("North", (0, 1)),
      ("Northeast", (1, 1)),
      ("Northwest", (-1, 1)),
      ("Gradient direction", None), # Will compute this one separately
  ]

  dV_dS1 = 100 + (50 / 20) * np.cos(s1 / 20) * np.cos(s2 / 20)
  dV_dS2 = 80 - (50 / 20) * np.sin(s1 / 20) * np.sin(s2 / 20)

  # The gradient direction (steepest ascent)
  grad_mag = np.sqrt(dV_dS1**2 + dV_dS2**2)

  results = []

  for name, direction in directions:
      if direction is None:
          # Use gradient direction
          direction = (dV_dS1, dV_dS2)

      d1, d2 = direction
      mag = np.sqrt(d1**2 + d2**2)
      u1, u2 = d1/mag, d2/mag

      D_u_V = dV_dS1 * u1 + dV_dS2 * u2

      results.append({
          'name': name,
          'direction': f"({d1:.2f}, {d2:.2f})",
          'unit': f"({u1:.3f}, {u2:.3f})",
          'D_u_V': D_u_V
      })

  return results

# Example usage
results = analyze_directions(100, 100)
for r in results:
  print(f"{r['name']}: D_u V = {r['D_u_V']:.2f}")

East: D_u V = 100.20
North: D_u V = 77.70
Northeast: D_u V = 125.80
Northwest: D_u V = -15.91
Gradient direction: D_u V = 126.80


### Hessian


$$H = \begin{pmatrix} \frac{\partial^2 V}{\partial S_1^2} & \frac{\partial^2 V}{\partial S_1 \partial S_2} \\ \frac{\partial^2 V}{\partial S_2 \partial S_1} & \frac{\partial^2 V}{\partial S_2^2} \end{pmatrix}$$

For this portfolio:

$$\frac{\partial^2 V}{\partial S_1^2} = - \frac{k}{400} \sin \left(\frac{S_1}{20}\right) \cos \left(\frac{S_2}{20}\right)$$

$$\frac{\partial^2 V}{\partial S_2^2} = - \frac{k}{400} \sin \left(\frac{S_1}{20}\right) \cos \left(\frac{S_2}{20}\right)$$

$$\frac{\partial^2 V}{\partial S_1 \partial S_2} = - \frac{k}{400} \cos \left(\frac{S_1}{20}\right) \sin \left(\frac{S_2}{20}\right)$$

In [55]:
import numpy as np

def compute_hessian(s1, s2, k=50):
  h11 = -k/400*np.sin(s1/20)*np.cos(s2/20)
  h22 = -k/400*np.sin(s1/20)*np.cos(s2/20)
  h12 = -k/400*np.cos(s1/20)*np.sin(s2/20)

  H = np.array([[h11, h12],
                [h12, h22]])
  return H

# Example usage
H = compute_hessian(100, 100)
print(f"Hessian at (100, 100):\n{H}")


Hessian at (100, 100):
[[0.03400132 0.03400132]
 [0.03400132 0.03400132]]


### Eigenvalues

For the principal *curvature*

In [56]:
import numpy as np

def analyze_hessian(H):
  eigenvalues, eigenvectors = np.linalg.eig(H)

  # Sort by eigenvalue magnitude (descending)
  idx = np.argsort(np.abs(eigenvalues))[::-1]
  eigenvalues = eigenvalues[idx]
  eigenvectors = eigenvectors[:, idx]

  return eigenvalues, eigenvectors

# Example usage
H = np.array([[0.034, 0.034], [0.034, 0.034]])
eigenvalues, eigenvectors = analyze_hessian(H)
print(f"Eigenvalues: λ₁ = {eigenvalues[0]:.4f}, λ₂ = {eigenvalues[1]:.4f}")
print(f"Eigenvector 1: {eigenvectors[:, 0]}")
print(f"Eigenvector 2: {eigenvectors[:, 1]}")

Eigenvalues: λ₁ = 0.0680, λ₂ = 0.0000
Eigenvector 1: [0.70710678 0.70710678]
Eigenvector 2: [-0.70710678  0.70710678]


#### Approximating the Surface

**First-Order (Linear) Approximation**

$$V(S_1, S_2) \approx V(S_1^*, S_2^*) + \frac{\partial V}{\partial S_1} (S_1 - S_1^*) + \frac{\partial V}{\partial S_2} (S_2 - S_2^*)$$

In [57]:
import numpy as np

# Portfolio value function
# V ≈ V* + ∂V/∂S₁(S₁ - S₁*) + ∂V/∂S₂(S₂ - S₂*)
def portfolio_value(s1, s2, w1=100, w2=80, k=50):
  linear_part = w1 * s1 + w2 * s2
  phi = k * np.sin(s1 / 20) * np.cos(s2 / 20)
  return linear_part + phi

def first_order_approximation(s1, s2, s1_star=100, s2_star=100):
  
  V_star = portfolio_value(s1_star, s2_star)

  dV_dS1 = 100 + (50 / 20) * np.cos(s1_star / 20) * np.cos(s2_star / 20)
  dV_dS2 = 80 - (50 / 20) * np.sin(s1_star / 20) * np.sin(s2_star / 20)

  delta_s1 = s1 - s1_star
  delta_s2 = s2 - s2_star

  V_approx=V_star+dV_dS1*delta_s1+dV_dS2*delta_s2

  return V_approx

# Example usage
s1_test, s2_test = 105, 98
V_true = portfolio_value(s1_test, s2_test)
V_approx = first_order_approximation(s1_test, s2_test)
print(f"True V(105, 98) = {V_true:.2f}")
print(f"Linear approx  = {V_approx:.2f}")
print(f"Error = {V_true - V_approx:.2f}")

True V(105, 98) = 18331.99
Linear approx  = 18332.00
Error = -0.01


### Second-Order (Quadratic) Approximation

Adding the Hessian term to improve accuracy:

$$V \approx V^* + \nabla V^T \Delta \mathbf{S} + \frac{1}{2} \Delta \mathbf{S}^T H \Delta \mathbf{S}$$

In [58]:
import numpy as np

# Portfolio value function
def portfolio_value(s1, s2, w1=100, w2=80, k=50):
  linear_part = w1 * s1 + w2 * s2
  phi = k * np.sin(s1 / 20) * np.cos(s2 / 20)
  return linear_part + phi

# First order approximation (from previous exercise)
def first_order_approximation(s1, s2, s1_star=100, s2_star=100):
  V_star = portfolio_value(s1_star, s2_star)
  dV_dS1 = 100 + (50 / 20) * np.cos(s1_star / 20) * np.cos(s2_star / 20)
  dV_dS2 = 80 - (50 / 20) * np.sin(s1_star / 20) * np.sin(s2_star / 20)
  delta_s1 = s1 - s1_star
  delta_s2 = s2 - s2_star
  return V_star + dV_dS1 * delta_s1 + dV_dS2 * delta_s2

def second_order_approximation(s1, s2, s1_star=100, s2_star=100):
  V_star = portfolio_value(s1_star, s2_star)

  # Gradient
  dV_dS1 = 100 + (50 / 20) * np.cos(s1_star / 20) * np.cos(s2_star / 20)
  dV_dS2 = 80 - (50 / 20) * np.sin(s1_star / 20) * np.sin(s2_star / 20)

  # Hessian
  h11 = -(50 / 400) * np.sin(s1_star / 20) * np.cos(s2_star / 20)
  h22 = -(50 / 400) * np.sin(s1_star / 20) * np.cos(s2_star / 20)
  h12 = -(50 / 400) * np.cos(s1_star / 20) * np.sin(s2_star / 20)

  delta_s1 = s1 - s1_star
  delta_s2 = s2 - s2_star

  # First order
  linear_term = dV_dS1 * delta_s1 + dV_dS2 * delta_s2

  quadratic_term = 0.5 * (h11 * delta_s1**2 + 2 * h12 * delta_s1 * delta_s2 + h22 * delta_s2**2)

  return V_star + linear_term + quadratic_term

# Compare approximations
s1_test, s2_test = 110, 95
V_true = portfolio_value(s1_test, s2_test)
V_linear = first_order_approximation(s1_test, s2_test)
V_quadratic = second_order_approximation(s1_test, s2_test)

print(f"True V(110, 95)     = {V_true:.2f}")
print(f"Linear approx       = {V_linear:.2f} (error: {abs(V_true - V_linear):.2f})")
print(f"Quadratic approx    = {V_quadratic:.2f} (error: {abs(V_true - V_quadratic):.2f})")

True V(110, 95)     = 18598.67
Linear approx       = 18599.91 (error: 1.23)
Quadratic approx    = 18600.33 (error: 1.66)
